## Build A Motion Detected Alarm System with Python and OpenCV

### Introduction

In this exercise we are going to develop – again! – a motion triggered alarm based on the frame differencing method. Now, we are going to do this using Python and OpenCV.

Open Source Computer Vision Library (OpenCV) is a library written in C/C++ mainly aimed at real-time computer vision.

OpenCV supports a wide variety of programming languages such as C++, Python, Java, etc. 

In particular, in this exercise we are going to introduce OpenCV-Python, a Python wrapper for the original OpenCV C++ implementation.


The approach for developing this application is the same as proposed in Exercise 14:

When the program starts, the program will capture from the webcam the ‘baseline’ image (see the left image below). This is the image without any 'intruder'. The program will keep capturing frames and comparing them with the baseline image. If no one enters the frame, nothing will happen. However, when somebody or something enters the frame (see the right image below), the program will detect that the captured frame and the baseline image are ‘very’ different and will trigger the audio alarm.

![Alarm](../Images/Alarm_v2.png)

### Exercises

**Very important: you need to run the code in your local machine for displaying the video and using the webcam.**

**Exercise 1.1**: Download (or copy) this Jupyter notebook into your local machine.

**Exercise 1.2**: Install the required Python libraries

                pip install pyttsx3
                pip install pywin32
                pip install numpy
                pip install opencv-python

**Exercise 1.3**: Run and study the code *Capture and display the frames of a video with Python and OpenCV*.

**Exercise 1.4**: Run and study the code *Capture and display webcam with Python and OpenCV*.

**Exercise 1.5**: Run and study the code *Build a motion detected alarm system with Python*.

### Exercise 1.3. Capture and display the frames of a video with Python and OpenCV.

 Run and study the code Capture and display the frames of a video with Python and OpenCV.

In [ ]:
!pip install pyttsx3
!pip install pywin32
!pip install numpy
!pip install opencv-python

Note: This lab is based on https://github.com/arindomjit/Motion_Detected_Alarm by Arindomjit Bhattacharjee.

In [2]:
import cv2

cap = cv2.VideoCapture('../../Downloads//Exercise1_Files/Traffic_Laramie_1.mp4')

# Create background subtractor object
fgbg = cv2.createBackgroundSubtractorMOG2()

# Capture the first frame and use it as the background image
_, bg = cap.read()
height, width , _ = bg.shape
bg = cv2.cvtColor(bg, cv2.COLOR_BGR2GRAY)

bg = bg[int(height/2): height, 0:width]


# Initialize list to hold trackers for each car
trackers = []

while True:
    # Read a new frame
    ret, frame = cap.read()
    if not ret:
        break
    height, width , _ = frame.shape
    main_street = frame[int(height/2): height, 0:width]
    
    # Convert the current frame to grayscale
    gray = cv2.cvtColor(main_street, cv2.COLOR_BGR2GRAY)
    
    
    # Perform background subtraction to obtain the foreground mask
    fgmask = fgbg.apply(gray)

    # Perform frame differencing to obtain the difference mask
    diff = cv2.absdiff(gray, bg)

    # Combine the foreground mask and the difference mask to get the final mask
    mask = cv2.bitwise_and(fgmask, diff)

    # Apply morphological operations to improve mask quality
#     kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
#     mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

    # Find contours in the final mask
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Loop over the contours to filter the cars and create bounding boxes
    for cnt in contours:
        if cv2.contourArea(cnt) < 2000:
            continue
        x, y, w, h = cv2.boundingRect(cnt)
        area = w * h
        aspect_ratio = w / h
        if area > 7000 and aspect_ratio > 0.3 and aspect_ratio < 3:


#         (x, y, w, h)=cv2.boundingRect(c)
            cv2.rectangle(frame, (x, y + int(height/2)), (x+w, y+h + int(height/2)), (0,255,0), 1)

#         # Filter based on size, aspect ratio, and position
#         if area > 500 and aspect_ratio > 0.3 and aspect_ratio < 3 and y > 100 and y < 400:
#             # Initialize a new tracker for the car
#             tracker = cv2.TrackerCSRT_create()
#             tracker.init(frame, (x, y, w, h))
#             trackers.append(tracker)

    # Update the tracker for each car and draw the bounding box
#     for tracker in trackers:
#         ok, bbox = tracker.update(frame)
#         if ok:
#             x, y, w, h = [int(v) for v in bbox]
#             cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Show the result
    cv2.imshow('Frame', frame)
    cv2.imshow('Mask', mask)
    cv2.imshow('Main street', main_street)

    # Update the background image for the next frame
    bg = gray

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

# traffic counter

In [6]:
import cv2

cap = cv2.VideoCapture('../../Downloads//Exercise1_Files/Traffic_Laramie_2.mp4')

# Create background subtractor object
fgbg = cv2.createBackgroundSubtractorMOG2()

# Capture the first frame and use it as the background image
_, bg = cap.read()
height, width , _ = bg.shape

x_offset = 150
y_offset = 170
x_start_point = int(width/2) - x_offset
x_end_point = int(width/2 )
y_start_point = int(height/2)
y_end_point = int(height/2) + y_offset
bg = cv2.cvtColor(bg, cv2.COLOR_BGR2GRAY)


bg = bg[y_start_point: y_end_point,x_start_point:x_end_point]

prev_loc = []
curr_loc = []
count = 0 
while True:
    # Read a new frame
    ret, frame = cap.read()
    if not ret:
        break
    height, width , _ = frame.shape
    city_center_street = frame[int(height/2): (int(height/2) + y_offset), int(width/2) - x_offset:int(width/2 )]
    
    # Convert the current frame to grayscale
    gray = cv2.cvtColor(city_center_street, cv2.COLOR_BGR2GRAY)
    
    # Perform background subtraction to obtain the foreground mask
    fgmask = fgbg.apply(gray)

    # Perform frame differencing to obtain the difference mask
    diff = cv2.absdiff(gray, bg)

    # Combine the foreground mask and the difference mask to get the final mask
    mask = cv2.bitwise_and(fgmask, diff)

    # Find contours in the final mask
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    curr_loc = []
    x = y = w = h= 0
   

    # Loop over the contours to filter the cars and create bounding boxes
    for cnt in contours:
        if cv2.contourArea(cnt) < 2000:
            continue
            
        x, y, w, h = cv2.boundingRect(cnt)
       
        area = w * h
        aspect_ratio = w / h
        if area > 6500 and aspect_ratio > 0.3 and aspect_ratio < 3:

            x=  x + x_start_point
            y = y + y_start_point
           
            cv2.rectangle(frame, (x, y ), (x+w, y+h), (0,255,0), 1)
            curr_loc.extend((x,y))
            
    if len(prev_loc) > 0 and len(curr_loc) == 0:
        print('car passed')
        count+=1


           
    cv2.rectangle(frame, (int((x_start_point + x_end_point)/2), y_start_point  ), (int((x_start_point + x_end_point)/2), y_end_point), (0,255,0), 1)
    # Show the result
    cv2.imshow('Frame', frame)
    cv2.imshow(' Counting point', city_center_street)
    cv2.putText(frame, f"Cars passed: {count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Update the background image for the next frame
    bg = gray
    prev_loc = curr_loc.copy()

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindos()




car passed
car passed
car passed
car passed


AttributeError: module 'cv2' has no attribute 'destroyAllWindos'